<a href="https://colab.research.google.com/github/muyale/Book-Recommender-System/blob/main/Book_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing some dependencies
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# loading in the data
df = pd.read_csv("/content/Books_llms.csv")

In [ ]:
df.head()

,Book-Title,Book-Author,Year-Of-Publication,Publisher,User Id
0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,1.0
1,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,2.0
2,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,3.0
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,4.0
4,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,5.0


In [4]:
df.columns

Index(['Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'User Id'],
      dtype='object')

In [ ]:
#df.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'],axis=1,inplace=True)

In [5]:
df.head()

,Book-Title,Book-Author,Year-Of-Publication,Publisher,User Id
0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,1.0
1,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,2.0
2,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,3.0
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,4.0
4,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,5.0


In [6]:
df.shape

(40110, 5)

In [7]:
df.isnull().sum()

Book-Title             0
Book-Author            1
Year-Of-Publication    1
Publisher              1
User Id                1
dtype: int64

In [8]:
df['Book-Author'] = df['Book-Author'].fillna('NotListed')

In [9]:
df['Publisher'] = df['Publisher'].fillna('NotListed')

In [10]:
df.isnull().sum()

Book-Title             0
Book-Author            0
Year-Of-Publication    1
Publisher              0
User Id                1
dtype: int64

In [11]:
# Getting important features
def get_important_features(df):
    important_features= []
    for i in range(df.shape[0]):
        important_features.append(df['Book-Title'][i]+df['Book-Author'][i]+df['Publisher'][i])
    return important_features

In [12]:
df_sample=df.head(6000)

In [13]:
df_sample['important_features'] = get_important_features(df_sample)

<ipython-input-13-307d2d6fe013>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['important_features'] = get_important_features(df_sample)


In [14]:
df_sample.columns

Index(['Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'User Id', 'important_features'],
      dtype='object')

In [15]:
df_sample.head(100)

,Book-Title,Book-Author,Year-Of-Publication,Publisher,User Id,important_features
0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,1.0,Classical MythologyMark P. O. MorfordOxford Un...
1,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,2.0,Clara CallanRichard Bruce WrightHarperFlamingo...
2,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,3.0,Decision in NormandyCarlo D'EsteHarperPerennial
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,4.0,Flu: The Story of the Great Influenza Pandemic...
4,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,5.0,The Mummies of UrumchiE. J. W. BarberW. W. Nor...
...,...,...,...,...,...,...
95,Pretend You Don't See Her,Mary Higgins Clark,1998.0,Pocket,96.0,Pretend You Don't See HerMary Higgins ClarkPocket
96,Fast Women,Jennifer Crusie,2001.0,St. Martin's Press,97.0,Fast WomenJennifer CrusieSt. Martin's Press
97,Female Intelligence,Jane Heller,2001.0,St. Martin's Press,98.0,Female IntelligenceJane HellerSt. Martin's Press
98,Pasquale's Nose: Idle Days in an Italian Town,Michael Rips,2002.0,Back Bay Books,99.0,Pasquale's Nose: Idle Days in an Italian TownM...


In [16]:
# Doing a TFIDF
tfidf = TfidfVectorizer()
relevant = tfidf.fit_transform(df_sample['important_features'])

In [17]:
# Using Cosine Similarity
c_similar = cosine_similarity(relevant)

indices = pd.Series(df_sample.index, index=df_sample['Book-Title'])


def get_recommendations(book_name):
    idx = indices[book_name]
    sim_scores = list(enumerate(c_similar[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:10]
    book_indices = [i[0] for i in sim_scores]
    return df_sample['Book-Title'].iloc[book_indices]

In [19]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.9 MB/s et

In [22]:
import gradio as gr
description = "TYPE IN YOUR FAVORITE BOOK "
title_trad = 'BOOK RECOMMENDER SYSTEM'
gr.Interface(fn=get_recommendations,inputs='text',outputs='text',title=title_trad,description=description).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4711b37d3ae21c9ff0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
